In [1]:
pip install transformers

In [2]:
!pip install imblearn

In [3]:
import torch
import numpy as np
import pandas as pd
import time
import re
from random import sample

from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

In [4]:
#Tempo de processamento
tempoInicial = time.time()

#Faz a leitura da base
df = pd.read_csv('dataset7DiasCompleto.v1.csv', sep=';')
df.describe()

colunaCorpus='titulo_processado'
colunaResultado='classe'

In [5]:
randomState = sample(range(0, 1000), 1)[0];

#Definindo X, y
X = df[colunaCorpus]
y = df[colunaResultado]

#Separa base treinamento e teste
XTreino, XTeste, yTreino, yTeste = train_test_split(X, y, train_size=0.7, 
                                                    stratify=y, shuffle=True, 
                                                    random_state=randomState)

#XTreino = XTreino[colunaCorpus].values
#yTreino = yTreino.values
#XTeste = XTeste[colunaCorpus].values
#yTeste = yTeste.values

In [6]:
#Balanceamento
#rus = RandomUnderSampler(random_state=randomState)
#XTreino, yTreino = rus.fit_resample(XTreino.values.reshape(-1,1), yTreino.values)

#XTreino = XTreino[:, 0]
#yTreino = yTreino.values
#XTeste = XTeste.values
#yTeste = yTeste.values

In [7]:
#Balanceamento
treino = pd.DataFrame(XTreino).join(yTreino)
qtde4 = treino[colunaResultado].value_counts()[4]

df0 = treino[treino[colunaResultado] == 0] #Separa a base de 0
df1 = treino[treino[colunaResultado] == 1] #Separa a base de 1
df2 = treino[treino[colunaResultado] == 2] #Separa a base de 2
df3 = treino[treino[colunaResultado] == 3] #Separa a base de 3
df4 = treino[treino[colunaResultado] == 4] #Separa a base de 4

#Under sampling
dfUnder0 = df0.sample(qtde4)
dfUnder1 = df1.sample(qtde4)
dfUnder2 = df2.sample(qtde4)
dfUnder3 = df3.sample(qtde4)
dfUnder = pd.concat([dfUnder0, dfUnder1, dfUnder2, dfUnder3, df4], axis=0)
dfShuffled = dfUnder.sample(frac=1).reset_index(drop=True)

XTreino = dfShuffled[colunaCorpus].values
yTreino = dfShuffled[colunaResultado].values
XTeste = XTeste.values
yTeste = yTeste.values

In [8]:
import transformers

## bert tokenizer
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [9]:
corpus = XTreino
#corpus = XTreino[colunaCorpus]
maxlen = 64

In [10]:
def tokenize(sentences, tokenizer):
    input_ids, input_masks, input_segments = [],[],[]
    for s in range(len(sentences)):
        sentence = sentences[s]
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=maxlen, pad_to_max_length=True, 
                                       return_attention_mask=True, return_token_type_ids=True,
                                       return_tensors = 'pt')
      
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])        

    return torch.cat(input_ids, dim=0), torch.cat(input_masks, dim=0), torch.cat(input_segments, dim=0)

X = tokenize(corpus, tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [11]:
import tensorflow as tf
from keras import layers
from keras import models

from transformers import BertModel

## pre-trained bert
model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states=False)
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [12]:
#Dados treinamento
with torch.no_grad():
    outputs = model(X[0], attention_mask=X[1])

#last layer
outputs[0].shape

cls_embeddings = outputs[0][:, 0, :].numpy()
cls_embeddings.shape

(3125, 768)

In [13]:
#all layers
#embeddings = outputs[2][12]
#len(embeddings)
#print(embeddings)

#def get_CLS_embedding(layer):
#    return layer[:, 0, :].numpy()
    
#cls_embeddings = []
#for i in range(12):
#    cls_embeddings.append(get_CLS_embedding(embeddings[i]))

#cls_embeddings[1]

In [14]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

#Define o classificador
classifier = RandomForestClassifier(class_weight="balanced")

#Define o scoring
score = 'accuracy'

#Validação cruzada com embaralhamento
kfold = StratifiedKFold(n_splits=10, shuffle=True)

#Grid Search
paramGrid = {
        'criterion': ['entropy', 'gini'],
        'n_estimators': np.arange(3, 101, 1).tolist(),
        'max_depth':  [None] + np.arange(1, 6, 2).tolist(),
        'min_samples_split': np.arange(2, 6, 1).tolist(),
        'min_samples_leaf': np.arange(1, 6, 1).tolist(),
        'min_weight_fraction_leaf': np.arange(0, 0.6, 0.1).tolist(),
        'max_features': ['auto'] + np.arange(0.1, 0.6, 0.1).tolist(),
        'bootstrap': [False, True],
        }

gridSearch = RandomizedSearchCV(estimator=classifier, param_distributions=paramGrid, cv=kfold, scoring=score, n_iter=10, n_jobs=-1)
gridSearch.fit(cls_embeddings, yTreino)

melhorEstimator = gridSearch.best_estimator_
classifier = melhorEstimator

In [15]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
    
arrayYReal = []
arrayYPrediction = []
arrayAcuracia = []

cv_iter = kfold.split(cls_embeddings, yTreino)
for treino, teste in cv_iter:
    #Etapa de treinamento
    classifier.fit(cls_embeddings[treino], yTreino[treino])

    #Etapa de predição
    yPrediction = classifier.predict(cls_embeddings[teste])

    arrayYReal = np.append(arrayYReal, yTreino[teste])
    arrayYPrediction = np.append(arrayYPrediction, yPrediction)

    arrayAcuracia.append(accuracy_score(yTreino[teste], yPrediction))

print(classification_report(arrayYReal, arrayYPrediction, labels=[0, 1, 2, 3, 4])) 

              precision    recall  f1-score   support

           0       0.67      0.69      0.68       625
           1       0.52      0.64      0.57       625
           2       0.69      0.57      0.62       625
           3       0.48      0.28      0.35       625
           4       0.56      0.75      0.64       625

    accuracy                           0.58      3125
   macro avg       0.58      0.58      0.57      3125
weighted avg       0.58      0.58      0.57      3125



In [16]:
corpus = XTeste
X = tokenize(corpus, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [17]:
#Dados teste

with torch.no_grad():
    outputs = model(X[0], attention_mask=X[1])

cls_embeddings = outputs[0][:, 0, :].numpy()
cls_embeddings.shape

(2594, 768)

In [18]:
## test
predicted = classifier.predict(cls_embeddings)
print(classification_report(yTeste, predicted, labels=[0, 1, 2, 3, 4]))

              precision    recall  f1-score   support

           0       0.63      0.71      0.67       452
           1       0.54      0.61      0.57       629
           2       0.73      0.57      0.64       645
           3       0.62      0.30      0.40       600
           4       0.35      0.76      0.48       268

    accuracy                           0.56      2594
   macro avg       0.57      0.59      0.55      2594
weighted avg       0.60      0.56      0.56      2594



In [19]:
print("\n--- %.2f minutos ---" % ((time.time() - tempoInicial) / 60))


--- 28.48 minutos ---
